In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
#pd.options.display.float_format = '{:.2f}'.format

# Load the data
data = pd.read_csv('../notebooks/data/merged_data.csv')

print(f'dataframe shape: {data.shape}')

In [ ]:
data.columns = data.columns.str.replace('survey_methamphetamine', 'survey_crystal')

list(data.columns)

In [ ]:

# create function to plot number of positive tests weekly for each drug class
def plot_drug_tests(df, type, drugclass, ax=None):
    """
    Plots the sum of drug test results for a specific drug class.

    Parameters:
    df (DataFrame): The input DataFrame containing the drug test data.
    type (str): The type of drug test, either 'test' or 'survey'.
    drugclass (str): The drug class to plot the results for.
    ax (Axes, optional): The matplotlib Axes object to plot the bar chart on. If not provided, a new Axes object will be created.

    Returns:
    ax (Axes): The matplotlib Axes object containing the bar chart.

    Raises:
    AssertionError: If the type is not 'test' or 'survey'.
    AssertionError: If the drugclass is not one of 'Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'.
    """

    # assert that type is either 'test' or 'survey'
    assert type in ['test', 'survey'], 'type must be either "test" or "survey"'

    # assert that drugclass must be include Propoxyphene, Amphetamines,Cannabinoids, Benzodiazepines, Cocaine (not case sensitive)
    assert drugclass in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'], 'drugclass must be one of "Propoxyphene", "Amphetamines", "Methamphetamine", "Cannabinoids", "Benzodiazepines", "Cocaine" or "Opiate300"'

    # filter df to total_visits == 26, this filters patients that completed treatment
    df = df[df['total_visits'] == 26]

    # create dataframe with columns for type and drug class
    df = df[[col for col in df.columns if col.startswith(type+'_') and drugclass in col]]

    # remove text and leave numbers in column names
    df.columns = [col.split('_')[-1] for col in df.columns]

    # fill nan values with 0
    df = df.fillna(0)

    # replace -5.0 with 0.0 
    df = df.replace(-5.0, 0.0)

    # plot the sum of the columns

    # set condition if axis object is not passed
    if ax is None:
        ax = plt.gca()
    sns.barplot(x=df.columns, y=df.sum(), ax=ax, palette='Blues_d')
    
    # create title
    ax.set_title(f' {drugclass} tests', fontsize=15)
    
    # remove the x axis 
    ax.set_xticklabels('')

    return ax

In [ ]:

# create function to plot number of positive tests weekly for each drug class
def drug_test_df(df, type, drugclass, ax=None):
    """
    Plots the sum of drug test results for a specific drug class.

    Parameters:
    df (DataFrame): The input DataFrame containing the drug test data.
    type (str): The type of drug test, either 'test' or 'survey'.
    drugclass (str): The drug class to plot the results for.
    ax (Axes, optional): The matplotlib Axes object to plot the bar chart on. If not provided, a new Axes object will be created.

    Returns:
    ax (Axes): The matplotlib Axes object containing the bar chart.

    Raises:
    AssertionError: If the type is not 'test' or 'survey'.
    AssertionError: If the drugclass is not one of 'Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'.
    """

    # assert that type is either 'test' or 'survey'
    assert type in ['test', 'survey'], 'type must be either "test" or "survey"'

    # assert that drugclass must be include Propoxyphene, Amphetamines,Cannabinoids, Benzodiazepines, Cocaine (not case sensitive)
    assert drugclass in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine', 'cannabis','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene','cannabis','cocaine','oxycodone','methadone','amphetamine','methamphetamine','opiates','benzodiazepines','propoxyphene'],f'{drugclass} not a valid drug class'

    # filter df to total_visits == 26, this filters patients that completed treatment
    df = df[df['total_visits'] == 26]

    # create dataframe with columns for type and drug class
    df = df[[col for col in df.columns if col.startswith(type+'_') and drugclass in col]]

    # remove text and leave numbers in column names
    df.columns = [col.split('_')[-1] for col in df.columns]

    # fill nan values with 0
    df = df.fillna(0)

    # replace -5.0 with 0.0 
    df = df.replace(-5.0, 0.0)

    df = df.sum()

    df = pd.DataFrame(df)

    df = df.rename(columns={0: f'{drugclass.lower()}'})

    return df

In [ ]:
group1 = data.loc[(data.responder==1)&(data.total_visits==26)]
group0 = data.loc[(data.responder==0)&(data.total_visits==26)]

In [ ]:
group1.shape[0], group0.shape[0]

In [ ]:
# create dataframe with number of drug test per drug class by treatment outcome

# these are the patients that responded to treatment

# create for loop to create a df for each drug class
for drug in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']:
 # turn each drug class into a dataframe
 globals()[f'{drug}'] = drug_test_df(group1, 'test', drug)

# merge all the dataframes
drug_test_r1 = pd.concat([Opiate300, Propoxyphene, Amphetamines, Methamphetamine, Cannabinoids, Benzodiazepines, Cocaine], axis=1)

# add '-r1' to the end of each column name
drug_test_r1.columns = [f'{col}_r1' for col in drug_test_r1.columns]

drug_test_r1

In [ ]:
# create dataframe with number of drug test per drug class by treatment outcome

# these are the patients that did not respond to treatment

# create for loop to create a df for each drug class
for drug in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']:
 # turn each drug class into a dataframe
 globals()[f'{drug}'] = drug_test_df(group0, 'test', drug)

# merge all the dataframes
drug_test_r0 = pd.concat([Opiate300, Propoxyphene, Amphetamines, Methamphetamine, Cannabinoids, Benzodiazepines, Cocaine], axis=1)

# add '_r1' to the end of each column name
drug_test_r0.columns = [f'{col}_r0' for col in drug_test_r0.columns]

drug_test_r0


In [ ]:
def calculate_test_pvalues(group1, group2):
    
    pval = []
    ci = []
    cd_list = []  
    bf10 = []
    power = []

    # perform ttest between each drug class
    for drug in ['opiate300','propoxyphene', 'amphetamines', 'methamphetamine', 'cannabinoids', 'benzodiazepines', 'cocaine']:
        t = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['p-val'][0]
        pval.append(t)
        c = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['CI95%'][0].tolist()
        c = ', '.join(map(str, c))
        ci.append(c)
        cd = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['cohen-d'][0]
        cd_list.append(cd)  
        b = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['BF10'][0]
        bf10.append(b)
        p = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['power'][0]
        power.append(p)

    test_pvs = pd.DataFrame(
        {'drugClass':['opiate300','propoxyphene', 'amphetamines', 'methamphetamine', 'cannabinoids', 'benzodiazepines', 'cocaine'],
         'pvalue':pval,
         'CI95%':ci,
         'cohen-d':cd_list,
         'BF10:':bf10,
         'power':power})

    return test_pvs

test_pvs = calculate_test_pvalues(drug_test_r1, drug_test_r0)

test_pvs

In [ ]:
# create loop to create a df for each drug class for survey data

for drug in ['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene']:
    # turn each drug class into a dataframe
    globals()[f'{drug}'] = drug_test_df(group1, 'survey', drug)
    
# merge all the dataframes
from functools import reduce

drug_survey_r1 = reduce(lambda x, y: pd.merge(x, y, right_index=True,left_index=True), [cannabis, cocaine, oxycodone, methadone, amphetamine, crystal, opiates, benzodiazepines, propoxyphene])

# add '-r1' to the end of each column name
drug_survey_r1.columns = [f'{col}_r1' for col in drug_survey_r1.columns]

drug_survey_r1

In [ ]:
# create loop to create a df for each drug class for survey data

for drug in ['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene']:
    # turn each drug class into a dataframe
    globals()[f'{drug}'] = drug_test_df(group0, 'survey', drug)
    
# merge all the dataframes
from functools import reduce

drug_survey_r0 = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), [cannabis, cocaine, oxycodone, methadone, amphetamine, crystal, opiates, benzodiazepines, propoxyphene])

# add '-r1' to the end of each column name
drug_survey_r0.columns = [f'{col}_r0' for col in drug_survey_r0.columns]

# rename columns
drug_survey_r0

drug_survey_r0

In [ ]:
# for responder group 1 -  avg reported instances of cocaine use over days 30 - 60 in treatment
(drug_survey_r1.iloc[1:2,1:2]/group1.shape[0]).round(2)

In [ ]:
# for responder group 0 -  avg reported instances of cocaine use over days 30 - 60 in treatment
(drug_survey_r0.iloc[1:2, 1:2]/group0.shape[0]).round(2)

In [ ]:
# for responder group 0 -  avg reported instances of cannabis use over days 0 - 30 in treatment
drug_survey_r0.iloc[1:2,:1]/group0.shape[0]

In [ ]:
# for responder group 1 -  avg reported instances of cannabis use over days 0 - 30 in treatment
drug_survey_r1.iloc[1:2,:1]/group1.shape[0]

In [ ]:
def calculate_survey_pvalues(group1, group2):
    
    tests = []
    ci = [] 
    cd_list = []
    bf10 = [] 
    power = []

    # perform ttest between each drug class
    for drug in ['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene']:
        t = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['p-val'][0]
        tests.append(t)
        c = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['CI95%'][0].tolist()
        c = ', '.join(map(str, c))
        ci.append(c)
        cd = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['cohen-d'][0]
        cd_list.append(cd)  
        b = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['BF10'][0]
        bf10.append(b)
        p = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['power'][0]
        power.append(p)

    survey_pvs = pd.DataFrame(
        {'drugClass':['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene'],
         # length of each column in group1 df
         'pvalue':tests,
         'CI95%':ci,
         'cohen-d':cd_list,
         'BF10:':bf10,
         'power':power})

    return survey_pvs

survey_pvs = calculate_survey_pvalues(drug_survey_r1, drug_survey_r0)

survey_pvs

In [ ]:
totalgroup = group1.shape[0] + group0.shape[0]

In [ ]:
# show confidence intervals for each drug class
ci = survey_pvs.loc[:,['CI95%']]

# separate the confidence intervals into two columns
ci = ci['CI95%'].str.split(', ', expand=True)

ci.columns = ['lower', 'upper']

ci = ci.astype(float)

ci['group_lower'] = ci['lower']/totalgroup
ci['group_upper'] = ci['upper']/totalgroup

# drugclass
ci['drugClass'] = survey_pvs['drugClass']

# move  drugclass to first column
cols = ci.columns.tolist()
cols = cols[-1:] + cols[:-1]
ci = ci[cols]

ci


